<a href="https://colab.research.google.com/github/viniciusrdo/workshop-engdados/blob/main/WorkshopEngDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
# Configurando as variáveis de ambiente para o Java e Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
# Instalando biblioteca do buscador e buscando a instalação do Spark
!pip install -q findspark
import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.2-bin-hadoop2.7'

In [7]:
# Importando e criando a SparkSession
# a SparkSession é o nosso ponto de entrada para as APIs do framework Spark

from pyspark.sql import SparkSession
sc = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Printando as informações da session criada
sc      

In [14]:
# Fazendo download do arquivo
!wget --verbose --show-progress --no-check-certificate https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv

--2022-10-10 22:47:12--  https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6216 (6.1K) [text/plain]
Saving to: ‘receita_federal_arrecadacao_por_UF_2020.csv.1’

receita_federal_arr 100%[===================>]   6.07K  --.-KB/s    in 0s      

2022-10-10 22:47:12 (55.2 MB/s) - ‘receita_federal_arrecadacao_por_UF_2020.csv.1’ saved [6216/6216]



In [15]:
# carregando um conjunto de dados que baixamos da internet
receitafederal = sc.read.csv(
    path = "/content/receita_federal_arrecadacao_por_UF_2020.csv", 
    inferSchema = True, 
    header = True,
    sep = ';', 
    encoding = "UTF-8")

In [16]:
# Verificando o tipo de objeto criado
type(receitafederal)

pyspark.sql.dataframe.DataFrame

In [17]:
# Espiando o dataset
receitafederal.show()

+---+------------+----+------------------------+------------------------+-------------+---------------------------+-------------+--------------+-------------------------------+-------------------------------+-------------------------+--------------+-----------------------------+--------------+-----------------+------------------------------------------------------+-----------------------------+
| uf|      regiao| ano|imposto_sobre_importacao|imposto_sobre_exportacao|    ipi_total|imposto_sobre_a_renda_total|         irpf|          irpj|imposto_s_renda_retido_na_fonte|imposto_s_operacoes_financeiras|imposto_territorial_rural|        cofins|contribuicao_para_o_pis_pasep|          csll|cide_combustiveis|cpsss_contrib_p_o_plano_de_segurid_social_serv_publico|outras_receitas_administradas|
+---+------------+----+------------------------+------------------------+-------------+---------------------------+-------------+--------------+-------------------------------+----------------------------

In [18]:
# Verificando o schema() deste sparkdataframe
receitafederal.printSchema()

root
 |-- uf: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- imposto_sobre_importacao: string (nullable = true)
 |-- imposto_sobre_exportacao: string (nullable = true)
 |-- ipi_total: string (nullable = true)
 |-- imposto_sobre_a_renda_total: string (nullable = true)
 |-- irpf: string (nullable = true)
 |-- irpj: string (nullable = true)
 |-- imposto_s_renda_retido_na_fonte: string (nullable = true)
 |-- imposto_s_operacoes_financeiras: string (nullable = true)
 |-- imposto_territorial_rural: string (nullable = true)
 |-- cofins: string (nullable = true)
 |-- contribuicao_para_o_pis_pasep: string (nullable = true)
 |-- csll: string (nullable = true)
 |-- cide_combustiveis: string (nullable = true)
 |-- cpsss_contrib_p_o_plano_de_segurid_social_serv_publico: string (nullable = true)
 |-- outras_receitas_administradas: string (nullable = true)



In [27]:
# importando os métodos com funções para transformações de variáveis
from pyspark.sql.functions import *

In [22]:
# Transformando o atributo irpf em numerica
receitafederal = receitafederal.withColumn(
    colName = 'irpf', 
    col = regexp_replace('irpf',',','.').cast('float')
    )

# Inspecionando o resultado
receitafederal.select('irpf').printSchema()

root
 |-- irpf: float (nullable = true)



In [21]:
# Verificando o total do irpf por Região do Brasil
receitafederal.groupBy('regiao').sum('irpf').orderBy('regiao').show()

+------------+--------------+
|      regiao|     sum(irpf)|
+------------+--------------+
|Centro-Oeste| 3.354157696E9|
|    Nordeste| 4.303029696E9|
|       Norte| 1.404179308E9|
|     Sudeste|2.496098528E10|
|         Sul| 7.380957184E9|
|       Total|4.140331008E10|
+------------+--------------+

